### Let's try finding Venus' ephemerides in the ICRF coordinate system.

In [24]:
from skyfield.api import load
from skyfield import units

# Create a timescale and initialise beginning and end time.
ts = load.timescale()
t0= ts.utc(2011,8,18,10,0,0)
t1= ts.utc(2013)

# Load the JPL ephemeris DE421 (covers 1900-2050).
planets = load('de421.bsp')
venus = planets['venus']
sun = planets['sun']

# print(planets)
# print(venus)


In [25]:
# The position of Venus, viewed from BCRS
venus_pos = venus.at(t0)
ra, dec, distance = venus_pos.radec() 

print(ra)
print(dec)
print(distance)
print(venus_pos)


09h 59m 15.86s
+15deg 40' 26.7"
0.72137 au
<Barycentric BCRS position and velocity at date t center=0 target=299>


In [27]:
# Let's convert this in degrees

import astropy.units as u

dec = dec.to(u.deg)
ra = ra.to(u.deg)

print("Position of Venus on the 18th august 2011 at 10am : ",dec, ra)

Position of Venus on the 18th august 2011 at 10am :  15.674088161331355 deg 149.81610096082233 deg


Ok, now that we know how to compute ephemerides of Venus in degrees in the ICRF, let's try to send a job to ODA to 
find an image of venus with IBIS. What's INTEGRAL center of coordinate system?

We know that IBIS has quite a large FoV so that serendipitous observations of Venus is possible. 
The goal now is to try find a moment when IBIS was observing a portion of the sky where Venus was.

Fully-coded IBIS FoV: 8.3°x8°. Down to zero: 29.1°x29.4°. What's the portion of the sky that this covers ? 
Whole sky is about 41'000 $deg^2$ so the FoV covers between 66.4 and 855.5 square degrees. 
The last number is quite elevated. Chances to find Venus are actually high I think.

What's Venus's apparent size in the sky on average ? Do we want to compute a probability of finding it ?
As in the 2001 paper, what could be interesting would be to find when Venus was close to its maximum elongation so that we have less 
pollution from the Sun.
